In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [30]:
import numpy as np

import pandas as pd
import sys, os, time
from scipy.stats import norm
import matplotlib.pyplot as plt

%matplotlib inline

sys.path.append('../')

from src.data_structures import FactorGraph, PriorityQueue
from src.sampling_algorithms.event_time_samplers import gaussian_bounce, chain_bounce_fn
from src.utils import gaussian_grad_potential_fn, gaussian_chain_grad_potential_fn, interp, get_xtv, get_first_moment, get_second_moment, get_var
from src.sampling_algorithms import LocalBPS
from src.plots.arrow_plot import arrow_plot

from arviz.stats import ess

from matplotlib import rc
rc('text', usetex=False)

In [337]:
d = 2
groups = 10
N = 100

def logistic(x):
    return 1/(1+np.exp(-x))

beta0 = np.random.randn(d)
betas = []
observations = []
covs = []

for g in range(groups):
    beta = beta0 + 2.*np.random.randn(d)
    betas.append(beta)
    
    cov = np.random.rand(d*N).reshape(N,d)*2.-1.
    covs.append(cov)
    p = logistic(cov.dot(beta))
    y = (np.random.random(len(p)) < p)+0.
    observations.append(y)

In [14]:
covs = np.array(covs)
observations = np.array(observations)

In [ ]:
cov_p = np.maximum(0., covs)
cov_n = -np.minimum(0., covs)
sign = np.expand_dims((-1.)**(observations), -1)

inds = [[d + d*i +j for j in range(d)] for i in range(groups)]

ind = inds[i]

v = init_v[ind]
x = init_x[ind]

def alias_sample(i, cov_n, cov_p, sign):
    s = sign[i]
    cp = cov_p[i]
    cn = cov_n[i]
    
    def bounce_fn(x,v):
        pos = (v*s)>=0
        neg = (v*s)<0
        ms = np.abs(v) * (pos*cp + neg*cn)
        m_i = np.sum(ms,0)
        m = np.sum(m_i)

        p = m_i/m

        k, = np.random.multinomial(1,p, size=1).ravel().nonzero()
        p = ms[:,k].flatten() / m_i[k]
        r, = np.random.multinomial(1,p, size=1).ravel().nonzero()
        
        u = np.random.rand()
        t = -np.log(u)/m

        if np.random.rand() < lambda_r(v,x,t,covs[i,r], y[r])/m:
            token = 'B'
        else:
            token = 'NB'
        return t, token
    return bounce_fn

In [ ]:
lambda_r(v,x,t,covariates, y)

In [285]:
obs_expand = np.expand_dims(observations, -1)
arr1 = cov_p*(obs_expand==1) + cov_n*(obs_expand==0) 
arr2 = cov_p*(obs_expand==0) + cov_n*(obs_expand==1) 

m_t = np.sum(arr2*np.abs(v), 1)
M_t = np.sum(m_t, 1, keepdims=True)

m_t/M_t

ValueError: operands could not be broadcast together with shapes (10,100,2) (10,2) 

In [111]:
m_t = np.sum(arr1*np.abs(v), 1)
M_t = np.sum(m_t, 1, keepdims=True)

m_t/M_t

array([[0.99469508, 0.00530492],
       [0.51583856, 0.48416144],
       [0.56191792, 0.43808208],
       [0.72289314, 0.27710686],
       [0.03203691, 0.96796309],
       [0.69026341, 0.30973659],
       [0.41853059, 0.58146941],
       [0.63270338, 0.36729662],
       [0.93840284, 0.06159716],
       [0.77463528, 0.22536472]])

In [216]:
def lambda_r(v,x,t,covariates, y):
    e = np.exp(covariates.dot(x))
    return np.maximum(0.,(covariates*(e/(1+e)-y)).dot(v))

def lambda_bound(v):
    return np.sum(np.abs(v))

def grad_logistic(covariates, obs):
    def grad_fn(x):
        e = np.exp(covariates.dot(x)).reshape(N,1)
        y = obs.reshape(N,1)
        return (covariates*e/(1+e)-y).sum(0)
    return grad_fn

covariates = covs[0]
y = observations[0]



def generate_logistic_bounce(covariates, y, limit=100, debug=False):
    def logistic_bounce(x,v):
        accepted = False
        count = 0
        while not accepted:
            count += 1
            lambda_sim = lambda_bound(v)
            u1 = np.random.rand()
            u2 = np.random.rand()
            t = -np.log(u1)/lambda_sim
            acc = lambda_r(v,x,t, covariates, y)/lambda_sim
            if debug:
                print(acc)
            if u2 < acc:
                accepted = True
            if count > limit:
                return float('inf')
        return t
    return logistic_bounce

def aggregate_bounce(bounce_fns):
    def bounce_fn(x,v):
        return np.min([fn(x,v) for fn in bounce_fns])
    return bounce_fn

In [43]:
mu0 = np.array([0. for _ in range(d)])
sig0 = np.diag([1. for _ in range(d)])
mu1 = np.array([0. for _ in range(d)])
mu2 = np.array([0.])
sig1 = np.diag([1. for _ in range(d)])
sig2 = np.array([[1.]])
sig12 = np.array([[0.5] for _ in range(d)])

global_event_samplers = []
local_event_samplers = []

for g in range(groups):
    global_event_samplers.append(chain_bounce_fn(mu1, mu2, sig1, sig2, sig12))
    covariates = covs[g]
    y = observations[g]
    bounce_fns = [generate_logistic_bounce(covariates[r], y[r]) for r in range(N)]
    bounce_fn = aggregate_bounce(bounce_fns)
    local_event_samplers.append(bounce_fn)

bounce_fns = [gaussian_bounce(mu0, sig0)] + global_event_samplers + local_event_samplers

grad_factor_potential_fns = [gaussian_grad_potential_fn(mu0, sig0)] + \
[gaussian_chain_grad_potential_fn(mu1, mu2, sig1, sig12, sig2) for _ in range(groups)] + \
[grad_logistic(covs[g], observations[g]) for g in range(groups)]


inv_sig2 = np.linalg.pinv(sig2)
transform = np.dot(sig12, inv_sig2)
sig_bar = sig1 - np.dot(transform, sig12.T)
inv_sig = np.linalg.inv(sig_bar)
dim_x1 = len(mu1)
mu = np.concatenate([mu1, mu2], 0)
t2 = np.concatenate([-transform, np.diag(np.repeat(1., dim_x1))], 1)


factor_indices = np.array([[i for i in range(d)]] + \
                 [[0]+[d + d*i +j for j in range(d)] for i in range(groups)] + \
                 [[d + d*i +j for j in range(d)] for i in range(groups)] )

factor_potential_fns = [lambda x: x for _ in grad_factor_potential_fns]

nodes = list(set(n for f in factor_indices for n in f ))


graph = FactorGraph(dim_x=len(nodes),
                          factor_indices=factor_indices,
                          factor_potential_fns=factor_potential_fns,
                          grad_factor_potential_fns=grad_factor_potential_fns)

<ipython-input-43-81f387d451b6>:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  factor_indices = np.array([[i for i in range(d)]] + \


In [44]:
init_x = np.random.randn(len(nodes))
init_v = np.random.randn(len(nodes))
local_bps = LocalBPS(init_x = init_x,
         init_v = init_v,
         factor_graph = graph,
         bounce_fns=bounce_fns,
         refresh_rate=0.1)

In [ ]:
plt.plot(np.random.randn(len(nodes)))
plt.plot(init_x)

start = time.time()
nsim= 5*10**4
results = local_bps.simulate(nsim)
res = results
print(time.time()-start)

In [ ]:
betas_flat = np.array(betas).flatten()
for i in range(groups+1):
    x1,v1,t1=get_xtv(res,i)
    if i < d:
        beta = beta0[i]
    else:
        beta = betas_flat[i-1]
    print('Beta {0}: {1}'.format(i, beta))
    print(get_first_moment(x1, v1, t1))

In [ ]:
x1,v1,t1=get_xtv(res,0)
x2,v2,t2=get_xtv(res,1)

In [ ]:
burnin = 0
plot_limit = 10000
fig_size = (10,10)
plt.rcParams["axes.edgecolor"] = "0.15"
plt.rcParams["axes.linewidth"]  = 1.25

font = {
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)


fig = plt.figure(figsize=fig_size,frameon =True)
left, bottom, width, height = 0.1, 0.1, 0.8, 0.8
ax = fig.add_axes([left, bottom, width, height])
ax.set_title("Local BPS")
ax.set_xlabel("$x_1$",fontsize='large', fontweight='bold')
ax.set_ylabel("$x_2$",fontsize='large', fontweight='bold')
arrow_plot(x1[burnin:burnin+plot_limit],x2[burnin:burnin+plot_limit], head_length=0.01,head_width=0.01)
fig.savefig('./local_bps.eps', format='eps', dpi=1200)



In [ ]:
x1[0], x2[0]

In [ ]:
x1[-1], x2[-1]

In [ ]:
beta0

In [ ]:
betas